# RapidAPI接口
RapidAPI 是一个 API 市场，提供了数千个 API，可以帮助开发人员快速找到并使用需要的 API。它包括多个类别，如人工智能、云计算、区块链、金融、游戏等，每个类别下面都有大量的 API 接口可供选择。在 RapidAPI 的平台上，你可以搜索、筛选、订阅和使用这些 API 接口，还可以查看 API 的文档和使用示例，从而更好地了解和使用它们。
我们利用Rapid API平台上bing搜索引擎提供的api来拿到浏览器上的最新数据，以此来作为大模型的新的数据来源。

In [1]:
import requests

RapidAPIKey = "4747ae4f1msh26b54cefc0a56f5p1f8284jsn46704b3264a2"

class DeepSearch:
    def search(query: str = ""):
        query = query.strip()

        #if query == "":
        #    return ""

        if RapidAPIKey == "":
            return "请配置你的 RapidAPIKey"

        
        url = "https://bing-web-search1.p.rapidapi.com/search"

        querystring = {"q":query,"mkt":"en-us","safeSearch":"Off","textFormat":"Raw","freshness":"Day"}

        headers = {
        "X-BingApis-SDK": "true",
        "X-RapidAPI-Key": "84747ae4f1msh26b54cefc0a56f5p1f8284jsn46704b3264a2",
        "X-RapidAPI-Host": "bing-web-search1.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)

        #print(response.json())
        data_list = response.json()['value']

        if len(data_list) == 0:
            print("没查到")
            return ""
        else:
            result_arr = []
            result_str = ""
            count_index = 0
            data_size=len(data_list)
            if data_size >4:
                for i in range(4):
                    item = data_list[i]
                    title = item["name"]
                    description = item["description"]
                    item_str = f"{title}: {description}"
                    result_arr = result_arr + [item_str]
            else:
                for i in range(data_size):
                    item = data_list[i]
                    title = item["name"]
                    description = item["description"]
                    item_str = f"{title}: {description}"
                    result_arr = result_arr + [item_str]
            result_str = "\n".join(result_arr)
            print("查到了")
            return result_str


# CustomLLM
   langchain 默认的模型是 OpenAI的ChatGPT。对于局域网应用来说，因为信息安全的要求私有数据不能出网关，所以需要搭本地模型。其实整个应用的硬件成本最高的就是 LLM 的部署，最经济的方式就是一个局域网一个类型的 LLM 统一部署一个，为了保障硬件的充分利用。LLM 和 Langchain 分开部署的最大好处就是灵活性，其实Langchain 已经是一个非常棒的设计样板了，langchain 只做资源整合，任何重存储和重计算的服务全部在远端部署，给 langchain 的应用留足生长的空间。
   使用 LangChain 可以方便使用 OpenAI 的模型借口，但是由于各种限制，导致使用有所不便，那么我们可以考虑使用自定义的本地模型，其核心在于构建 langchain.llms.base.LLM 的子类 CustomLLM 并重写_call 函数如下：

In [2]:
from typing import Any, List, Dict, Mapping, Optional
import json

from langchain.callbacks.manager import CallbackManagerForLLMRun
import requests
from langchain.llms.base import LLM
from langchain.callbacks.base import Callbacks
from transformers import AutoModel,AutoTokenizer


class CustomLLM(LLM):

    logging: bool = False
    output_keys: List[str] = ["output"]

    llm_type: str = "Llama-2-7b"

    @property
    def _llm_type(self) -> str:
        return self.llm_type

    def log(self, log_str):
        if self.logging:
            #print(log_str)
        else:
            return

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        
        response = requests.post(f'http://10.1.36.75:8080/', {
        "ask": prompt
        })
        response=response.text
        return response

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"n": 10}

# CustomPromptTemplate

而对于控制推理决策和汇总答案两个动作到底怎么拼装不同的 Prompt，必须要定义新的 CustomPromptTemplate 类，他继承了 StringPromptTemplate。其实只要重写一个 format 函数就行，langchain 会自动把每次 LLM 返回作为参数传递给 format 函数。我只要判断哪是首次向 LLM 询问决策，哪是后续的拼装答案。

In [ ]:
from langchain.agents import Tool
from langchain.tools import BaseTool
from langchain import PromptTemplate, LLMChain
from langchain.agents import BaseSingleActionAgent, AgentOutputParser, LLMSingleActionAgent, AgentExecutor
from typing import List, Tuple, Any, Union, Optional, Type
from langchain.schema import AgentAction, AgentFinish
from langchain.prompts import StringPromptTemplate
from langchain.callbacks.manager import CallbackManagerForToolRun
import re

agent_template = """
你现在是一个{role}。这里是一些已知信息：
{related_content}
{background_infomation}
{question_guide}：{input}

{answer_format}
"""

class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool]
    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        if len(intermediate_steps) == 0:
            background_infomation = "\n"
            role = "傻瓜机器人"
            question_guide = "我现在有一个问题"
            answer_format = """请你只回答\"DeepSearch("搜索词")\"，并将"搜索词"替换为你认为需要搜索的关键词，除此之外不要回答其他任何内容。\n\n下面请回答我上面提出的问题！"""
        else:
            background_infomation = "\n\n你还有这些已知信息作为参考：\n\n"
            action, observation = intermediate_steps[0]
            background_infomation += f"{observation}\n"
            role = "聪明的 AI 助手"
            question_guide = "请根据这些已知信息回答我的问题"
            answer_format = ""
        kwargs["background_infomation"] = background_infomation
        kwargs["role"] = role
        kwargs["question_guide"] = question_guide
        kwargs["answer_format"] = answer_format
        return self.template.format(**kwargs)

class CustomSearchTool(BaseTool):
    name: str = "DeepSearch"
    description: str = ""

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None):
        return DeepSearch.search(query = query)

    async def _arun(self, query: str):
        raise NotImplementedError("DeepSearch does not support async")

class CustomAgent(BaseSingleActionAgent):
    @property
    def input_keys(self):
        return ["input"]

    def plan(self, intermedate_steps: List[Tuple[AgentAction, str]],
            **kwargs: Any) -> Union[AgentAction, AgentFinish]:
        return AgentAction(tool="DeepSearch", tool_input=kwargs["input"], log="")

class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        match = re.match(r'^[\s\w]*(DeepSearch)\(([^\)]+)\)', llm_output, re.DOTALL)
        if not match:
            return AgentFinish(
                return_values={"output": llm_output.strip()},
                log=llm_output,
            )
        else:
            action = match.group(1).strip()
            action_input = match.group(2).strip()
            return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)


class DeepAgent:
    tool_name: str = "DeepSearch"
    agent_executor: any
    tools: List[Tool]
    llm_chain: any
    def query(self, related_content: str = "", query: str = ""):
        tool_name = self.tool_name
        result = self.agent_executor.run(related_content=related_content, input=query ,tool_name=self.tool_name)
        return result

    def __init__(self, **kwargs):
        llm = CustomLLM()
        tools = [
                    Tool.from_function(
                        func=DeepSearch.search,
                        name="DeepSearch",
                        description=""
                    )
                ]
        self.tools = tools
        tool_names = [tool.name for tool in tools]
        output_parser = CustomOutputParser()
        prompt = CustomPromptTemplate(template=agent_template,
                                      tools=tools,
                                      input_variables=["related_content","tool_name", "input", "intermediate_steps"])

        llm_chain = LLMChain(llm=llm, prompt=prompt)
        self.llm_chain = llm_chain

        agent = LLMSingleActionAgent(
            llm_chain=llm_chain,
            output_parser=output_parser,
            stop=["\nObservation:"],
            allowed_tools=tool_names
        )

        agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
        self.agent_executor = agent_executor


if __name__ == "__main__":

    llm = CustomLLM()
    tools = [
                Tool.from_function(
                    func=DeepSearch.search,
                    name="DeepSearch",
                    description=""
                )
            ]
    tool_names = [tool.name for tool in tools]
    output_parser = CustomOutputParser()
    prompt = CustomPromptTemplate(template=agent_template,
                                  tools=tools,
                                  input_variables=["related_content","tool_name", "input", "intermediate_steps"])

    llm_chain = LLMChain(llm=llm, prompt=prompt)

    agent = LLMSingleActionAgent(
        llm_chain=llm_chain,
        output_parser=output_parser,
        stop=["\nObservation:"],
        allowed_tools=tool_names
    )

    agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
    print("欢迎使用Llama-2-7b模型，输入内容即可进行对话，clear清空对话记录，stop终止程序：")
    while True:
        query=input("\n用户：")
        print(agent_executor.run(related_content="", input=query, tool_name="DeepSearch"))

欢迎使用Llama-2-7b模型，输入内容即可进行对话，clear清空对话记录，stop终止程序：
